<a href="https://colab.research.google.com/github/poo5zan/llm_public/blob/main/Extract_Information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extract Information from company's private data using open source Large Language Models. In this document, I have used Llama3 8 billion model, which is an open source model from Meta (Facebook).

Important Note: Please make sure that the connected runtime has GPU. By default, the runtime might not have GPU. So, in order to change the runtime, click on the 'Runtime' menu, then click 'Change runtime type', and select 'T4 GPU'. The runtime type should be 'Python 3'.

# Ollama Installation

In [1]:
# Download and Start Ollama
# The command to start ollama i.e 'ollama serve' has been started in a new process.
# If you just run the command !ollama serve, then it will run the process in the main UI thread,
# thus blocking everything. You can try that too, and then revert back to this process method
!curl https://ollama.ai/install.sh | sh
import subprocess
process_serve = subprocess.Popen("ollama serve", shell=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10975    0 10975    0     0  29015      0 --:--:-- --:--:-- --:--:-- 28957
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


# Google Drive

Any files uploaded to the colab runtime will be deleted with each new session, and uploading the documents again and again is tedious. Thus, I prefer using google drive to store the documents. The following command will open a new window to connect to the google drive.

In [2]:
# connect google drive and mount the root folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


I created a folder named "llm_data" in my google drive as the root folder. You can name it anything.

TIP: After you utilize all the compute resources for free tier, then you can try with new gmail account. In case of google drive, you need to share this llm_data folder with the new gmail account, and then add shortcut to this folder in the root folder i.e 'My Drive' of the new google account. This way you don't need to re-upload your documents.

In [4]:
import os
llm_data_folder = "/content/drive/My Drive/llm_data"
print('llm_data_folder ', llm_data_folder)

llm_data_folder  /content/drive/My Drive/llm_data


# Install Python Dependencies


In [5]:
%pip install llama-index llama-parse ollama llama-index-llms-ollama  llama-index-llms-huggingface \
llama-index-embeddings-huggingface llama-index-extractors-entity transformers torch accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 7.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using c

In this document, we will be using llama3 both from Ollama and HuggingFace. The llama3 in HuggingFace is not public, thus you need to request access the owner of the model in the HuggingFace. It's a simple process, create an account in HuggingFace and request access to this model, https://huggingface.co/meta-llama/Meta-Llama-3-8B

In the following command, we login to the huggingface cli. This part is optional if you are only going to use llama3 from Ollama.